In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from keras.layers import Dense, LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dropout
from keras.layers import BatchNormalization
pd.options.mode.chained_assignment = None
tf.random.set_seed(0)

# download the data
df = pd.read_csv('ETH-CAD_nw.csv')
y = df['Close'].fillna(method='ffill')
y = y.values.reshape(-1, 1)

# scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(y)
y = scaler.transform(y)

# generate the input and output sequences
n_lookback = 1500  # length of input sequences (lookback period)
n_forecast = 30  # length of output sequences (forecast period)

X = []
Y = []

for i in range(n_lookback, len(y) - n_forecast + 1):
    X.append(y[i - n_lookback: i])
    Y.append(y[i: i + n_forecast])

X = np.array(X)
Y = np.array(Y)

# fit the model
#model = Sequential()
#model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lookback, 1)))
#model.add(LSTM(units=50))
#model.add(Dense(n_forecast))
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_lookback, 1)))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(BatchNormalization())  # Adding batch normalization for stability
model.add(LSTM(units=64))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(n_forecast))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, Y, epochs=100, batch_size=32, verbose=0)

# generate the forecasts
X_ = y[- n_lookback:]  # last available input sequence
X_ = X_.reshape(1, n_lookback, 1)

Y_ = model.predict(X_).reshape(-1, 1)
Y_ = scaler.inverse_transform(Y_)

# organize the results in a data frame
df_past = df[['Close']].reset_index()
df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True)
df_past['Date'] = pd.to_datetime(df['Date'])
#print(df_past['Date'])
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')

display(results.tail(31))
# plot the results
results.plot(title='AAPL')